In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
tf.__version__

'1.14.0'

### Data Preprocessing

In [2]:
VOCAB_SIZE = 5000
WIN_SIZE = 2

In [3]:
# load the movie comment dataset

imdb = keras.datasets.imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=VOCAB_SIZE)

In [4]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [5]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [6]:
decode_review(train_data[1])

"<START> big hair big <UNK> bad music and a giant safety <UNK> these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an <UNK> the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are <UNK> and funny in equal <UNK> the hair is big lots of <UNK> <UNK> men wear those cut <UNK> <UNK> that show off their <UNK> <UNK> that men actually wore them and the music is just <UNK> trash that plays over and over again in almost every scene there is trashy music <UNK> and <UNK> taking away bodies and the <UNK> still doesn't close for <UNK> all <UNK> aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [8]:
from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.utils import np_utils

In [15]:
def gen_context_word_pairs(corpus, window_size):
    context_length = window_size * 2
    for sentence in corpus:
        sen_len = len(sentence)
        for idx, word in enumerate(sentence):
            context = []
            target = []
            start = idx - window_size
            end = idx + window_size + 1
            # the context and the target word, we would use them to construct pair latter
            context.append([sentence[i] 
                             for i in range(start, end)
                             if 0 <= i < sen_len
                             and i != idx])
            target.append(word)
            x = sequence.pad_sequences(context, maxlen = context_length)
            y = target
            yield (x, y)

In [22]:
i = 0
for x, y in gen_context_word_pairs(corpus=train_data, window_size=WIN_SIZE):
    print(decode_review(x[0]), "|", decode_review(y))
    if i == 10:
       break
    i += 1

<PAD> <PAD> this film | <START>
<PAD> <START> film was | this
<START> this was just | film
this film just brilliant | was
film was brilliant casting | just
was just casting location | brilliant
just brilliant location scenery | casting
brilliant casting scenery story | location
casting location story direction | scenery
location scenery direction everyone's | story
scenery story everyone's really | direction


### Model Construction

In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda